In [1]:
from pystac import Item
from odc.stac import load
import folium
from ipyleaflet import basemaps

In [2]:
row = 63
col = 20
year = 2024

url = (
    "https://dep-public-staging.s3.us-west-2.amazonaws.com/dep_s2_seagrass/0-0-1/"
    f"{row:03}/{col:03}/{year}/dep_s2_seagrass_{row:03}_{col:03}_{year}.stac-item.json"
)
item = Item.from_file(url)

data = load([item])

# data

In [3]:
row = 52
col = 20
year = 2024

In [4]:
try:
    url = (
        "https://dep-public-staging.s3.us-west-2.amazonaws.com/dep_s2_seagrass/0-0-1/"
        f"{row:03}/{col:03}/{year}/dep_s2_seagrass_{row:03}_{col:03}_{year}.stac-item.json"
    )
    item = Item.from_file(url)
    data = load([item])
except Exception as e:
    print(f"Could not load tile row={row}, col={col}, year={year}: {e}")

In [5]:
data

<xarray.Dataset> Size: 277MB
Dimensions:               (y: 9600, x: 9600, time: 1)
Coordinates:
  * y                     (y) float64 77kB -2.08e+06 -2.08e+06 ... -1.984e+06
  * x                     (x) float64 77kB 1.992e+06 1.992e+06 ... 2.088e+06
    spatial_ref           int32 4B 3832
  * time                  (time) datetime64[ns] 8B 2024-01-01
Data variables:
    classification        (time, y, x) uint8 92MB 1 1 1 1 1 1 1 ... 1 1 1 1 1 1
    seagrass              (time, y, x) uint8 92MB 255 255 255 ... 255 255 255
    seagrass_probability  (time, y, x) uint8 92MB 255 255 255 ... 255 255 255

In [10]:
center = flipped.odc.geobox.geographic_extent.centroid.coords[0][::-1]

m = folium.Map(location=center, zoom_start=10, tiles=basemaps.Esri.WorldImagery)

for variable in data.data_vars:
    flipped.odc.add_to(m, name=variable, colormap="viridis")

folium.LayerControl().add_to(m)

m

# Please note the esri map tiles are not always aligned to data outputs here it is best to export your tifs below to see how they align with satellite imagery and the GeoMAD 

In [11]:
# flipped = data.classification.isel(y=slice(None, None, -1))  # Flips the y axis
# center = flipped.odc.geobox.geographic_extent.centroid.coords[0][::-1]
# m = folium.Map(location=center, zoom_start=10, tiles=basemaps.Esri.WorldImagery)

# flipped.odc.add_to(m, name=variable, colormap="viridis")

In [12]:
import boto3

bucket = "dep-public-staging"
prefix = "dep_s2_seagrass/0-0-1/"

s3 = boto3.client("s3")
result = s3.list_objects_v2(Bucket=bucket, Prefix=prefix)

tiles = []
for obj in result.get("Contents", []):
    key = obj["Key"]
    if key.endswith(".stac-item.json") and f"/{year}/" in key:
        # Parse row and col from the filename or path
        # Example: dep_s2_seagrass/0-0-1/064/033/2024/dep_s2_seagrass_064_033_2024.stac-item.json
        parts = key.split("/")
        row, col = parts[-3], parts[-2]
        tiles.append((int(row), int(col)))

In [13]:
# print(tiles)

In [14]:
data

<xarray.Dataset> Size: 277MB
Dimensions:               (y: 9600, x: 9600, time: 1)
Coordinates:
  * y                     (y) float64 77kB -2.08e+06 -2.08e+06 ... -1.984e+06
  * x                     (x) float64 77kB 1.992e+06 1.992e+06 ... 2.088e+06
    spatial_ref           int32 4B 3832
  * time                  (time) datetime64[ns] 8B 2024-01-01
Data variables:
    classification        (time, y, x) uint8 92MB 1 1 1 1 1 1 1 ... 1 1 1 1 1 1
    seagrass              (time, y, x) uint8 92MB 255 255 255 ... 255 255 255
    seagrass_probability  (time, y, x) uint8 92MB 255 255 255 ... 255 255 255

In [15]:
data = data.compute()

from odc.geo.xr import write_cog

write_cog(data.classification, "marine_habitats.tif", overwrite=True)
write_cog(data.seagrass, "seagrass.tif", overwrite=True)
write_cog(data.seagrass_probability, "seagrass_probability.tif", overwrite=True)

PosixPath('seagrass_probability.tif')